## Project

![](https://dl.dropboxusercontent.com/u/75194/dbase.jpg)

- custom database server
- postgres metadata server
- flask based (or other) http REST server
- hiding behind a standard web server such as nginx
- running on AWS
- with a html/javascript based UI

HTML Javascript based UI

- query using metadata
- similarity search amongst results of query
- or on any result from any search

## Web and Database Servers

- A server is a long running process (also called daemon) 
- which listens on a pre-specified port
- and responds to a request, which is sent using a
- protocol, such as HTTP or SMTP or something custom like the communication between a DBAPI2 driver and a database


### What does an API need?

The answer to this question goes to the root of you system design.

- Systems are designed around nouns and verbs.
- a noun is a class, or a module
- a verb is a method, or a function

- often APIs may be all verbs, especially in code
- indeed the idea behind an ABC (protocol) like "Sequence" is to collect these verbs
- these verbs might provide access to a noun, and usually

### HTTP is the lingua franca of the web

A browser must first we must parse the url. Everything after a `#` is a fragment. Until then its the DNS name or ip address, followed by the URL.

Our notebooks also talk to a local web server on our machines:

`http://localhost:8888/notebooks/cs207/FALL/lectures/ADT.ipynb#/slide-2-0`

- protocol is `http`
- hostname is `localhost`
- port is `8888`
- url is /notebooks/cs207/FALL/lectures/ADT.ipynb
- url fragment is #/slide-2-0

Now the protocol comes into action

Request:

`GET /request-URI HTTP/version`

In [19]:
import socket
def fetch(host, url):
    sock = socket.socket()
    sock.connect((host, 80))
    request = 'GET {} HTTP/1.0\r\nHost: {}\r\n\r\n'.format(url, host)
    print("Request is:\n", request)
    sock.send(request.encode('ascii'))
    response = b''
    chunk = sock.recv(4096)
    while chunk:
        response += chunk
        chunk = sock.recv(4096)
    return response

Response (from http://www.garshol.priv.no/download/text/http-tut.html)

```
HTTP/1.0 200 OK
Server: Netscape-Communications/1.1
Date: Tuesday, 25-Nov-97 01:22:04 GMT
Last-modified: Thursday, 20-Nov-97 10:44:53 GMT
Content-length: 6372
Content-type: text/html

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<HTML>
...followed by document content...

```

In [20]:
fetched = fetch("www.google.com", '/')
print('--------------', type(fetched))
print(fetched.decode('ISO-8859-1'))

Request is:
 GET / HTTP/1.0
Host: www.google.com


-------------- <class 'bytes'>
HTTP/1.0 200 OK
Date: Mon, 14 Nov 2016 04:49:02 GMT
Expires: -1
Cache-Control: private, max-age=0
Content-Type: text/html; charset=ISO-8859-1
P3P: CP="This is not a P3P policy! See https://www.google.com/support/accounts/answer/151657?hl=en for more info."
Server: gws
X-XSS-Protection: 1; mode=block
X-Frame-Options: SAMEORIGIN
Set-Cookie: NID=90=gb5q7b0_nF1LV3AO5gi67zVIoOIJY2B51iOYnrjlMuffay4yeZIbBpV6qTiDk7WJi-RY4IaLmrjjIyahotvfyhY5gILqCFQjAbeQ83LGwk8dsVMqsRB9yH2n6XvBgMmvzz-tPaPBRTZLAmU; expires=Tue, 16-May-2017 04:49:02 GMT; path=/; domain=.google.com; HttpOnly
Accept-Ranges: none
Vary: Accept-Encoding

<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="en"><head><meta content="Search the world's information, including webpages, images, videos and more. Google has many special features to help you find exactly what you're looking for." name="description"><meta content="noodp" na

### Making a dynamic web site using Flask

One could write static web pages. That's by far the fastest. (Indeed cached will serve rendered pages to you).

But if you want an app, at some point you have to embrace dynamicity.

Install `flask, sqlalchemy, flask-sqlalchemy`

In [58]:
%%file runner.sh
export FLASK_APP=$1
export FLASK_DEBUG=1
flask run

Overwriting runner.sh


In [60]:
%%file app_simple.py
from flask import Flask

app = Flask(__name__)

# view


@app.route('/')
def index():
    return '<h1>Hello Evil World!</h1>'

if __name__ == '__main__':
    app.run()

Overwriting app_simple.py


`pip install wtforms"

In [62]:
%%file app_simple_1.py
from flask import Flask, request, abort, redirect, url_for, render_template, flash

app = Flask(__name__)
app.secret_key = 'secret key'
#forms

from wtforms import Form, StringField, validators

class TestForm(Form):
    username = StringField('Username', [validators.Length(min=1, max=25)])

# view


@app.route('/', methods=['GET'])
def index():
    username = request.args.get('username')
    if username is not None:
        rstring = '<h1> Hello '+username+'</h1>'
    else:
        rstring =  '<h1>Hello World!</h1>'
    return rstring

@app.route('/hello')
def hello():
    abort(404)

@app.route('/index')
def index2():
    return redirect(url_for('index'))

@app.route('/testform', methods=['GET', 'POST'])
def testform():
    form = TestForm(request.form)
    if request.method == 'POST' and form.validate():
        flash("Thanks for testing %s" % form.username)
        return redirect(url_for('index')+'?username='+form.username.data)
    return render_template('testform.html', form=form)

if __name__ == '__main__':
    app.run()

Overwriting app_simple_1.py


In [63]:
%%file templates/testform.html
{% from "_formhelpers.html" import render_field %}
<form method="post" action="/testform">
  <dl>
    {{ render_field(form.username) }}
  </dl>
  <p><input type="submit" value="Test">
</form>


Writing templates/testform.html


In [64]:
%%file templates/_formhelpers.html

{% macro render_field(field) %}
  <dt>{{ field.label }}
  <dd>{{ field(**kwargs)|safe }}
  {% if field.errors %}
    <ul class=errors>
    {% for error in field.errors %}
      <li>{{ error }}</li>
    {% endfor %}
    </ul>
  {% endif %}
  </dd>
{% endmacro %}

Writing templates/_formhelpers.html


### HTTP Status Codes

I reproduce the really important ones:

(from same link)

`200 OK`
>Means that the server did whatever the client wanted it to, and all is well.

` 201 Created`
>The request has been fulfilled and resulted in a new resource being created. The newly created resource can be referenced by the URI(s) returned in the entity of the response, with the most specific URI for the resource given by a Location header field. 

`400: Bad request`
>The request sent by the client didn't have the correct syntax.

`401: Unauthorized`
>Means that the client is not allowed to access the resource. This may change if the client retries with an authorization header.

`403: Forbidden`
>The client is not allowed to access the resource and authorization will not help.

`404: Not found`
>Seen this one before? :) It means that the server has not heard of the resource and has no further clues as to what the client should do about it. In other words: dead link.

`500: Internal server error`
>Something went wrong inside the server.

`501: Not implemented`
>The request method is not supported by the server.

### Communicating on the Web

Why only users? Why not machines?

In the old hoary days, endpoints on servers were run using xml-rpc. Then came SOAP and the involvement of industry heavyweights and one standard after another, solving problems that didnt exist.


#### xmlrpc

xmlrpc is a Remote Procedure Call idea (RPC)

Thus, xml-rpc is very verb oriented. Its very useful when you dont want to define a protocol and are not latency limited. See the python standard library docs.

In [25]:
from xmlrpc.client import ServerProxy, Error

# server = ServerProxy("http://localhost:8000") # local server
with ServerProxy("http://betty.userland.com", verbose=True) as proxy:
    print(proxy)

    try:
        print(proxy.examples.getStateName(41))
    except Error as v:
        print("ERROR", v)

<ServerProxy for betty.userland.com/RPC2>
send: b'POST /RPC2 HTTP/1.1\r\nHost: betty.userland.com\r\nAccept-Encoding: gzip\r\nContent-Type: text/xml\r\nUser-Agent: Python-xmlrpc/3.5\r\nContent-Length: 161\r\n\r\n'
send: b"<?xml version='1.0'?>\n<methodCall>\n<methodName>examples.getStateName</methodName>\n<params>\n<param>\n<value><int>41</int></value>\n</param>\n</params>\n</methodCall>\n"
reply: 'HTTP/1.1 200 OK\r\n'
header: Connection header: Content-Length header: Content-Type header: Date header: Server body: b'<?xml version="1.0"?>\r\n<methodResponse>\r\n\t<params>\r\n\t\t<param>\r\n\t\t\t<value>South Dakota</value>\r\n\t\t\t</param>\r\n\t\t</params>\r\n\t</methodResponse>\r\n'
South Dakota


In [28]:
(b"<?xml version='1.0'?>\n<methodCall>\n<methodName>examples.getStateName</methodName>\n<params>\n<param>\n<value><int>41</int></value>\n</param>\n</params>\n</methodCall>\n").decode()

"<?xml version='1.0'?>\n<methodCall>\n<methodName>examples.getStateName</methodName>\n<params>\n<param>\n<value><int>41</int></value>\n</param>\n</params>\n</methodCall>\n"

### REST (Representational state transfer)

From wikipedia:

>The term is intended to evoke an image of how a well-designed web application behaves: It is a network of web resources (a virtual state-machine) where the user progresses through the application by selecting links, such as /user/tom, and operations such as GET or DELETE (state transitions), resulting in the next resource (representing the next state of the application) being transferred to the user for their use.

- web resources are modeled as nouns
- operations are stateless, as in functional programming: "no client context being stored on the server between requests"
- endpoints accept and return requests in XML, HTML, JSON
- From wikipedia: 
>Each request from any client contains all the information necessary to service the request, and session state is held in the client. The session state can be transferred by the server to another service such as a database to maintain a persistent state for a period and allow authentication. The client begins sending requests when it is ready to make the transition to a new state. While one or more requests are outstanding, the client is considered to be in transition. The representation of each application state contains links that may be used the next time the client chooses to initiate a new state-transition.

#### What is REST?

(https://blog.miguelgrinberg.com/post/designing-a-restful-api-with-python-and-flask)

The characteristics of a REST system are defined by six design rules:

**Client-Server**: There should be a separation between the server that offers a service, and the client that consumes it.

**Stateless**: Each request from a client must contain all the information required by the server to carry out the request. In other words, the server cannot store information provided by the client in one request and use it in another request.

**Cacheable**: The server must indicate to the client if requests can be cached or not.

**Layered System**: Communication between a client and a server should be standardized in such a way that allows intermediaries to respond to requests instead of the end server, without the client having to do anything different.

**Uniform Interface**: The method of communication between a client and a server must be uniform.

**Code on demand**: Servers can provide executable code or scripts for clients to execute in their context. This constraint is the only one that is optional.

#### Example of products

In [68]:
%%file flask_rest.py
from flask import Flask, request, abort, redirect, url_for, jsonify, make_response

app = Flask(__name__)

# models

the_products = [
    dict(kind = "toy", name = "jim", sku=1),
    dict(kind = "book", name = "Harry Potter", sku=2),
    dict(kind = "toy", name = "buzz", sku=3),
    dict(kind = "book", name = "Learning from Data", sku=4)
]



# view


@app.route('/products', methods=['GET'])
def products():
    kind = request.args.get('kind', '')
    if kind:
        prods=[s for s in the_products if s['kind']==kind]
        return jsonify(dict(products=prods))
    else:
        return jsonify(dict(products=the_products))

@app.route('/products/<int:sku>', methods=['GET'])
def product(sku):
    prods=[s for s in the_products if s['sku']==sku]
    if len(prods) == 0:
        abort(404)
    return jsonify({'product': prods[0]})

@app.route('/products', methods=['POST'])
def make_product():
    print(request)
    if not request.json or not 'name' in request.json:
        abort(400)
    prod = {
        'sku': the_products[-1]['sku'] + 1,
        'name': request.json['name'],
        'kind': request.json.get('kind', "toy")
    }
    the_products.append(prod)
    return jsonify({'product': prod}), 201


@app.errorhandler(404)
def not_found(error):
    return make_response(jsonify({'error': 'Not found'}), 404)

if __name__ == '__main__':
    app.run()


Overwriting flask_rest.py


Lets add a new product:

In [50]:
import json, requests

url = 'http://localhost:5000/products'
payload = {'name': 'Dory'}
r = requests.post(url, json=payload)

In [51]:
r.json()

{'product': {'kind': 'toy', 'name': 'Dory', 'sku': 5}}

### Real persistence: a database

The persistence above will only last until the next time i reload the server...

The usual way to manage persistence in these situations is to use a database..and it can be used for permanent items, such as products, or ephemeral ones, such as sessions and cookies corresponding to them..(although this contradicts REST).

We'll populate the database directly using DBAPI2, with code taken from lab9:


In [56]:
%%file populate_db.py
# models

the_products = [
    dict(kind = "toy", name = "jim", sku=1),
    dict(kind = "book", name = "Harry Potter", sku=2),
    dict(kind = "toy", name = "buzz", sku=3),
    dict(kind = "book", name = "Learning from Data", sku=4)
]

from sqlite3 import dbapi2 as sq3
import os
PATHSTART="/tmp"
def get_db(dbfile):
    sqlite_db = sq3.connect(os.path.join(PATHSTART, dbfile))
    return sqlite_db

def init_db(dbfile, schema):
    """Creates the database tables."""
    db = get_db(dbfile)
    db.cursor().executescript(schema)
    db.commit()
    return db

schema = """
DROP TABLE IF EXISTS "products";
CREATE TABLE "products" (
    "sku" INTEGER PRIMARY KEY  NOT NULL ,
    "name" VARCHAR NOT NULL,
    "kind" VARCHAR
);
"""

import sys
db=init_db("products.db", schema)
ins="""
INSERT INTO products (sku, name, kind) \
    VALUES (?,?,?);
"""
for d in the_products:
    db.cursor().execute(ins, (d['sku'], d['name'], d['kind']))
db.commit()
db.close()


Writing populate_db.py


And then we'll use SQLAlchemy's ORM to make queries and add new products, since it makes things much simpler for us as object-oriented programmers

In [57]:
%%file flask_rest2.py

from flask import Flask, request, abort, redirect, url_for, jsonify, make_response
from flask.ext.sqlalchemy import SQLAlchemy, DeclarativeMeta
from json import JSONEncoder

class ProductJSONEncoder(JSONEncoder):

    def default(self, obj):
        if isinstance(obj.__class__, DeclarativeMeta):
            return obj.to_dict()
        return super(ProductJSONEncoder, self).default(obj)

app = Flask(__name__)
app.json_encoder = ProductJSONEncoder
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:////tmp/products.db'
db = SQLAlchemy(app)

class Product(db.Model):
    __tablename__ = 'products'
    sku = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(80), nullable=False)
    kind = db.Column(db.String(80))

    def __init__(self, name, kind):
        self.name = name
        self.kind = kind

    def __repr__(self):
        return '<User %r>' % self.name

    def to_dict(self):
        return dict(sku=self.sku, name=self.name,
                kind = self.kind)

# view


@app.route('/products', methods=['GET'])
def products():
    kind = request.args.get('kind', '')
    if kind:
        prods=Product.query.filter_by(kind=kind)
        return jsonify(dict(products=prods.all()))
    else:
        return jsonify(dict(products=Product.query.all()))

@app.route('/products/<int:sku>', methods=['GET'])
def product(sku):
    prods=Product.query.filter(sku=sku)
    if len(prods) == 0:
        abort(404)
    return jsonify({'product': prods.first()})

@app.route('/products', methods=['POST'])
def make_product():
    print(request)
    if not request.json or not 'name' in request.json:
        abort(400)
    prod = Product(name=request.json['name'], kind=request.json.get('kind', "toy"))
    db.session.add(prod)
    db.session.commit()
    return jsonify({'product': prod}), 201


@app.errorhandler(404)
def not_found(error):
    return make_response(jsonify({'error': 'Not found'}), 404)


if __name__ == '__main__':
    app.run()


Writing flask_rest2.py
